In [2]:
# Recursive training example selection for entity resolution
#
# Copyright 2015 Peter Christen, Dinusha Vatsalan and Qing Wang
# Email: peter.christen@anu.edu.au
# Web:   http://users.cecs.anu.edu.au/~christen/
#
#   This program is free software: you can redistribute it and/or modify
#   it under the terms of the GNU General Public License as published by
#   the Free Software Foundation, either version 3 of the License, or
#   (at your option) any later version.
#
#   This program is distributed in the hope that it will be useful,
#   but WITHOUT ANY WARRANTY; without even the implied warranty of
#   MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
#   GNU General Public License for more details.
#
#   You should have received a copy of the GNU General Public License
#   along with this program.  If not, see <http://www.gnu.org/licenses/>.
#
# -----------------------------------------------------------------------------
#
# TODO: - allow several weight vectors per corner in select_corners function
#       - what to do with clusters which are pure but too large
#         (> max_cluster_size), how to split? (they need to be split)
#
# Usage:
#
# python recursive-train-selection.py [weight_vector_file] [min_data_set_size]
#                                     [weights_to_use] [init_method]
#                                     [select_method] [sample_error]
#                                     [oracle_acc] [split_classifier]
#                                     [min_cluster_size] [max_cluster_size]
#                                     [min_purity] [budget_num_class]
#                                     [res_file_name]
# where:
#
# weight_vector_file  The name of the weight vector file, with an assumed
#                     format of: [rec_id1,rec_id2,true_match_status,w1,w2..]
#                     where: rec_id1/rec_id2 are the record identifiers,
#                            true_match is 1.0 if the weight vector was
#                              generated by a true matching record pair,
#                              0.0 otherwise,
#                            w1, w2, ..., wd the d weights (normalised into
#                              0..1)
# min_data_set_size   The number of records in the smaller of the two data
#                     sets, used to calculate the initial proportion of
#                     matches.
#
# weights_to_use      A list of numbers of the weights to use. Must be of
#                     the format [x,y,z] with x/y/z>=0 and x/y/z<=(d-1) and no
#                     spaces between numbers.
#
# init_method         How to select the initial weight vectors, can either be
#                     'far' (select weight vectors farthest away from each
#                     other), '01' (select weight vectors closest to [0,..,0]
#                     and [1,..,1], 'corner' (select weight vectors closest to
#                     all corners of the multi-dimensional space), and 'ran'
#                     for a random selection.
#
# select_method       Which approach to use when selecting representative
#                     weight vectors in a cluster, possible are:
#                       'far' (only use farthest away weight vectors), or
#                       'far_med' (also include medoid weight vector)
#                       'dense' (density based)
#                       'aggl'  (agglomerative clustering based)
#                       'ran' (random selection)
#
# sample_error        When calculating number of samples to use for a certain
#                     cluster, the margin of error required. Must be a value
#                     between 0.0 and below 1.0 (typical 0.05 to 0.2)
#
# oracle_acc          The assumed accuracy of the oracle a value between 0.5
#                     and 1.
# 
# split_classifier    The approach used to classify and split matches from
#                     non-matches, possible are:
#                       'knn'   (for k nearest neighbour classifier)
#                       'svm'   (for support vector machine classifier)
#                       'dtree' (for decision tree classifier)
#
# min_cluster_size    The minimum size of a cluster allowed, clusters are not
#                     split further below that size. Must be a positive number.
#
# max_cluster_size    The maximum size of a cluster allowed before it is added
#                     to the training set. Must be a positive number larger
#                     than min_cluster_size.
#
# min_purity          The minimum 'purity' (i.e. accuracy) of a cluster, once
#                     this is reached a cluster will not be split further. Must
#                     be a value between 0.5 and 1.
#
# budget_num_class    The number of manual classifications we can do by the
#                     oracle.
#
# queue_order         The method used to order the queue of clusters and
#                     decide which cluster to process next. Possible are:
#                     - 'fifo'      First in first out, our initial approach
#                                   (PAKDD'15)
#                     - 'random'    Randomly select a cluster from the queue
#                     - 'max_puri'  Cluster with highest purity first (based
#                                   on purity of parent cluster, as child
#                                   cluster purity can only be larger)
#                     - 'min_puri'  Clusters with lowest purity first (again
#                                   based on parent cluster purity)
#                     - 'min_entr'  Clusters with lowest entropy first (based 
#                                   on entropy of parent cluster, as child
#                                   cluster entropy can only be smaller)
#                     - 'max_entr'  Clusters with highest entropy first (again
#                                   based on parent cluster entropy)
#                     - 'max_size'  Largest clusters first
#                     - 'min_size'  Smallest clusters first
#                     - 'close_01'  Closest to 0 or 1 corner
#                     - 'close_mid' Closest to middle (half between 0 and 1
#                                   corners)
#                     - 'balance'   Select so that training set sizes are
#                                   balanced
#                     - 'sample'    Select cluster which has largest ratio of
#                                   cluster size divided by number of number
#                                   of samples required
#
# res_file_name       Name of the file where results are to be appended.

# TODO: Maybe add: ****************************
# fuzzy_reg_ratio     A value between 0 and 1 giving the maximum ratio of the
#                     fuzzy region from where not to select weight vectors in
#                     the splitting phase (i.e. if distances between matches
#                     and non-matches are nearly the same (idea adapted from:
#                     http://link.springer.com/chapter/10.1007/11677437_12,
#                     Section 4.2)

# -----------------------------------------------------------------------------
#
import gzip
import itertools
import random
import math
import os, errno
import sys
import time

import numpy
import sklearn.svm
import sklearn.tree
import sklearn.cluster

import csv
import copy

random.seed(42)

# -----------------------------------------------------------------------------
# Define some constant values
 
# Which column in weight vector file contains match status
#
# TRUE_MATCH_COL = 2
TRUE_MATCH_COL = 6 #Após a próxima coluna é que ficam os vetores de similaridade

# Maximum allowed pureness before a weight vector is completely removed from
# input weight vector set (if larger then only the minority 'class' weight
# vectors are removed)
#
MIN_REMOVE_PURENESS = 0.1

# k-nn classifier k value
#
KNN_K = 7

# Number of weight vectors to sample (per cluster) if a set is too large in the
# agglomerative selection approach
#
NUM_SAMPLE = 100

RUN_SVM = False  # If the final SVM classifers are to be run or not

# Which distance calculation to use for cluster minimums
#
CLUSTER_MIN_DIST = 'average'  # One of: 'single', 'average', 'complete'

# -----------------------------------------------------------------------------
# Process command line arguments
#
#weight_vec_file_name = sys.argv[1]
weight_vec_file_name = 'vetorSimilaridades-14-03.csv'
# weight_vec_file_name = 'vetorSimilaridades-DEMO.csv'

#min_data_set_size = int(sys.argv[2])
min_data_set_size = 1295
assert min_data_set_size > 1

#weights_to_use = sys.argv[3]  # List of weights to use (format: [i,j,.. ,k])
# weights_to_use = '[0,1,2,3,4]'  # List of weights to use (format: [i,j,.. ,k])
# weights_to_use = '[0,1,2,3,4,5,6]'  # List of weights to use (format: [i,j,.. ,k])
weights_to_use = '[0,1,2,3]'  # List of weights to use (format: [i,j,.. ,k])
weights_to_use_list = eval(weights_to_use)
num_weights = len(weights_to_use_list)

for i in range(num_weights):  # Convert into indices in weight vector lists
  assert weights_to_use_list[i] >= 0
#   weights_to_use_list[i] += 3 #A partir da 4ª coluna (coluna 3)
  weights_to_use_list[i] += 7 #A partir da 8ª coluna (coluna 7) de vetor_similaridade

#init_method = sys.argv[4]
init_method = 'far'
assert init_method in ['far', '01', 'corner', 'ran']

#select_method = sys.argv[5]
select_method = 'far'
assert select_method in ['far', 'far_med', 'ran', 'dense', 'aggl']

#sample_error = float(sys.argv[6])
sample_error = 0.1
assert sample_error > 0.0 and sample_error < 1.0

#oracle_acc = float(sys.argv[7])
oracle_acc = 1.0
assert oracle_acc > 0.5 and oracle_acc <= 1.0

#split_classifier = sys.argv[8]
split_classifier = 'svm'
assert split_classifier in ['knn', 'svm', 'dtree']

#min_cluster_size = int(sys.argv[9])
min_cluster_size = 20
assert min_cluster_size >= 1

#max_cluster_size = int(sys.argv[10])
max_cluster_size = 50
assert max_cluster_size > min_cluster_size

#min_purity = float(sys.argv[11])
min_purity = 0.95
assert min_purity > 0.5 and min_purity < 1.0

#budget_num_class = int(sys.argv[12])
# budget_num_class = 1178 # Tamanho máximo de PC
budget_num_class = 200 # Vou deixar assim por enquanto
assert budget_num_class >= 1

#queue_order = sys.argv[13]
# queue_order = 'random'
queue_order = 'balance'
assert queue_order in ['fifo', 'random', 'max_puri', 'min_puri', 'max_size',
                       'min_size', 'min_entr', 'max_entr', 'close_01',
                       'close_mid', 'balance', 'sample']

#res_file_name = sys.argv[14]
res_file_name = 'resultado.csv'

weight_vector_dict_orig = {}

# -----------------------------------------------------------------------------
# Function to load weight vector file
#
def load_weight_vec_file(in_file_name, weights_to_use_list):
  """Load weight vector file, return a dictionary of weight vectors (as lists)
     of the form:
                 [rec_id1,rec_id2,true_match_status,w1,w2,...,wd]
     where only the selected weights in the 'weights_to_use_list' are being
     kept.

     Returns a list of the names of these weights (from header line) as well
     as the list of weight vectors, where each weight vector is a tuple of
     weights.
  """

  print
  print 'Load weight vector file:', in_file_name

  # Load weight vector file, extract weights from selected attributes
  #
  weight_vector_dict = {}  # Keys will be record identifiers

  num_dup_rec_ids = 0  # Number of record IDs (antity ID pairs) that occur
                       # several times

  if (in_file_name.endswith('.gz')):
    in_file = gzip.open(in_file_name)
  else:
    in_file = open(in_file_name)
  header_line = in_file.readline()
  #header_list = header_line.strip().split(',') #Remove todos os espaços em branco e divide a partir das vírgulas
  
  header_list = header_line.strip().split(';') #Remove todos os espaços em branco e divide a partir das vírgulas

  weights_name_list = []
  for w in weights_to_use_list:
    weights_name_list.append(header_list[w])
  print '  Weights to use:', weights_name_list

  for line in in_file: #Para cada linha no arquivo
    line_list = line.strip().split(';') #Lista com todas as linhas
    #print line_list
    #rec_id =    line_list[0].strip()+'-'+line_list[1].strip() #Armazena os identificadores dos pares!
    rec_id =    line_list[0].strip()+'~'+line_list[1].strip() #Armazena os identificadores dos pares!

    # Check if a certain entity-ID pair occurs more than once
    #
    if (rec_id in weight_vector_dict):
      num_dup_rec_ids += 1
    assert line_list[TRUE_MATCH_COL] in ['0.0', '1.0']

    if (line_list[TRUE_MATCH_COL] == '1.0'):
      match_status = True
    else:
      match_status = False

    this_weight_vec = []
    for w in weights_to_use_list:
      this_weight_vec.append(float(line_list[w].strip()))

    weight_vector_dict[rec_id] = (match_status, tuple(this_weight_vec))

  print '  Number of weight vectors:', len(weight_vector_dict)
  print '    Number of entity ID pairs that occurred more than once:', \
        num_dup_rec_ids 
  print
  
  #Adicionado por mim
  
  #lst = weight_vector_dict.items() 
  
  #for i in [1,4,10]:
	#print lst[i] 
	
  sub = 0.1889763779527559
  sub2 = 0.5487804878048781
     
    
  for k,v in weight_vector_dict.iteritems(): #v[0] corresponde à classe; 
											 #v[1] corresponde a uma tupla com o vetor de pesos
    if (sub in v[1]) & (sub2 in v[1]):
        print k
  #Isso é retornado:
	#REC_ID-422-REC_ID-230
	#REC_ID-425-REC_ID-217
	#REC_ID-425-REC_ID-215
	#REC_ID-423-REC_ID-230
	#REC_ID-424-REC_ID-230
	  
  #print weight_vector_dict['REC_ID-1~REC_ID-0']
  
  

  return weights_name_list, weight_vector_dict

# -----------------------------------------------------------------------------
# Function to analyse the quality of a dictionary of weight vectors, and then
# remove weight vectors of low quality (i.e. those that correspond to both
# matches and non-matches).
#
def analyse_filter_weight_vectors(weight_vector_dict):
  """Find number of unique weight vectors, their frequencies, and their
     pureness (i.e. how many are matches/non-matches).

     Returns the modified weight vector dictionary with non-pure weight vectors
     removed, a dictionary with all unique weight vectors, as well as the
     number of true matches and non-matches in the original weight vector
     dictionary.
  """

  num_weight_vectors = len(weight_vector_dict)

  print 'Analyse set of %d weight vectors' % (num_weight_vectors)

  unique_weight_vec_dict = {}  # Keys are weight vectors, values counts of how
                               # of these are matches and non-matches
  num_true_matches =     0  # Count the number of true matches and non-matches
  num_true_non_matches = 0

  for rec_id in weight_vector_dict:
    (match_status, weight_vector_tuple) = weight_vector_dict[rec_id]

    match_count_list = unique_weight_vec_dict.get(weight_vector_tuple, [0,0])
    if (match_status == True):
      num_true_matches += 1
      match_count_list[0] += 1
    else:
      num_true_non_matches += 1
      match_count_list[1] += 1

    unique_weight_vec_dict[weight_vector_tuple] = match_count_list

  num_unique_weight_vectors = len(unique_weight_vec_dict)

  print '  Containing %d true matches and %d true non-matches' % \
        (num_true_matches, num_true_non_matches)
  print '    (%.2f%% true matches)' % \
        (100.0*num_true_matches / len(weight_vector_dict))

  print '  Identified %d unique weight vectors' % (num_unique_weight_vectors)

  count_dict = {}  # Counts of how often weight vectors occur
  for match_count_list in unique_weight_vec_dict.itervalues():
    weight_vec_count = sum(match_count_list)
    sum_count = count_dict.get(weight_vec_count, 0) + 1
    count_dict[weight_vec_count] = sum_count

  count_list = count_dict.items()
  count_list.sort()

  print '  Frequency distribution of occurences of weight vectors:'
  print '    Occurence : Number of weight vectors that occur that often'
  for (freq, count) in count_list:
    print '      %5d : %5d  (%.2f%%)' % \
          (freq, count, 100.0*count/num_unique_weight_vectors)
  print

  # Identify all non-pure weight vectors and remove from original dictionary of
  # weight vectors
  #
  non_pure_weight_vec_dict = {}  # Keys will be weight vector tuples, values
                                 # their pureness
  pureness_dict = {}  # Also collect statistics

  for (weight_vector_tuple, match_count_list) in \
                                           unique_weight_vec_dict.iteritems():
    pureness = float(match_count_list[0]) / sum(match_count_list)
    pureness_count = pureness_dict.get(pureness, 0) + 1
    pureness_dict[pureness] = pureness_count

    if (pureness not in [0.0, 1.0]):
      non_pure_weight_vec_dict[weight_vector_tuple] = pureness

  pureness_list = pureness_dict.items()
  pureness_list.sort(reverse=True)

  print 'Identified %d non-pure unique weight vectors' % \
        (len(non_pure_weight_vec_dict)), '(from %d unique weight vectors)' % \
        (len(unique_weight_vec_dict))

  print 'Pureness (as percentage of matches) for a certain unique weight ' + \
        'vector:'
  print '  Pureness : Count'
  for (pureness, count) in pureness_list:
    print '     %5.3f : %2d' % (pureness, count),
    if (pureness not in [0.0, 1.0]):
      if ((pureness < MIN_REMOVE_PURENESS) or \
          (pureness > (1.0- MIN_REMOVE_PURENESS))):
        print '  (minority class weight vectors with this pureness to be ' + \
              'removed)'
      else:
        print '  (all weight vectors with this pureness to be removed)'
    else:
      print
  print
  
  # Remove non-pure weight vectors from the original dictionary of weight
  # vectors (if pureness worse than MIN_REMOVE_PURENESS then remove any
  # weight vector, otherwise only remove minority 'class' weight vectors
  #
  for rec_id in weight_vector_dict.keys():
    match_status, weight_vector_tuple = weight_vector_dict[rec_id]

    if weight_vector_tuple in non_pure_weight_vec_dict:
      weight_vector_pureness = non_pure_weight_vec_dict[weight_vector_tuple]

      # Check if the weight vector has to be removed in any case
      #
      if ((weight_vector_pureness > MIN_REMOVE_PURENESS) and \
          (weight_vector_pureness < (1.0-MIN_REMOVE_PURENESS))):
        del weight_vector_dict[rec_id]

      else:  # Only remove if the weight vector is in the minority 'class'

        # Mostly non-matches, so only remove weight vectors which are true
        # matches
        #
        if (weight_vector_pureness <= MIN_REMOVE_PURENESS):
          if (match_status == True):
            del weight_vector_dict[rec_id]

        else:  # Mostly matches, so only remove true non-matches
          if (match_status == False):
            del weight_vector_dict[rec_id]

  print 'Removed %d non-pure weight vectors' % \
        (num_weight_vectors - len(weight_vector_dict))
  print

  # Generate a weighted dictionary of unique weight vectors, their counts, and
  # their match status
  #
  weighted_unique_weight_vec_dict = {}  # Keys are weight vectors, values are
                                        # counts of how often they occur and
                                        # their match status

  for rec_id in weight_vector_dict:
    (match_status, weight_vector_tuple) = weight_vector_dict[rec_id]

    if (weight_vector_tuple not in weighted_unique_weight_vec_dict):
      weight_vector_count_match_list = [1, match_status]
    else:
      weight_vector_count_match_list = \
                          weighted_unique_weight_vec_dict[weight_vector_tuple]
      weight_vector_count_match_list[0] += 1
      assert weight_vector_count_match_list[1] == match_status
    weighted_unique_weight_vec_dict[weight_vector_tuple] = \
                                                weight_vector_count_match_list

  print 'Final number of weight vectors to use:', len(weight_vector_dict)
  print '  Number of unique weight vectors:', \
        len(weighted_unique_weight_vec_dict)
  print

  return weight_vector_dict, weighted_unique_weight_vec_dict, \
         num_true_matches, num_true_non_matches

# -----------------------------------------------------------------------------
# Function to calculate Euclidean distance between two vectors
#
def euclidean_dist(vec1, vec2, vec_len):

  dist = 0.0

  for i in range(vec_len):
    x = vec1[i] - vec2[i]
    dist += x*x

  return math.sqrt(dist)

# -----------------------------------------------------------------------------
# Function to convert a weight vector into a string with a specific number of
# digits (with 3 as default)
#
def weight_vector_to_str(weight_vector, num_digit=3):
  weight_vector_str = '['
  for w in weight_vector:
    digit_str = str(round(w, num_digit))
    if (len(digit_str.split('.')[-1]) < num_digit):
      num_miss_zero = num_digit - len(digit_str.split('.')[-1])
      digit_str += '0'*num_miss_zero

    weight_vector_str = weight_vector_str+digit_str+', '
  weight_vector_str = weight_vector_str[:-2]+']'

  return weight_vector_str

# -----------------------------------------------------------------------------
# Function to select weight vectors closest to [0,..,0] and [1,..,1] corners
#
def select_01(weight_vec_dict, k, num_weights):
  """Return k selected weight vectors from the given weight vector dictionary
     that are closest to the  [0,..,0] and [1,..,1] corners, with k/2 selected
     in each corner (if k is odd one more weight vector is selected in the
     [1,...,1] (match) corner.

     Returns a new dictionary that contains k weight vectors.
  """

  print 'Initial selection of weight vectors closest to [0,..,0] and [1,..,1]'

  zero_vector = [0.0]*num_weights
  one_vector =  [1.0]*num_weights

  # Two lists with tuples (distance to 0/1 and record identifier)
  #
  zero_close_list = []  # List of weight vectors closest to [0,..,0]
  one_close_list =  []  # List of weight vectors closest to [1,..,1]

  # Calculate length of the two lists
  #
  zero_list_len = k/2
  if (k % 2 == 0):
    one_list_len =  k/2
  else:
    one_list_len =  k/2+1

  print '  Select %d weight vectors closest to zero and %d closest to one.' % \
        (zero_list_len, one_list_len)

  max_0_dist = -1.0  # Current maximum distance to 0
  max_1_dist = -1.0  # Current maximum distance to 1

  for weight_vector_tuple in weight_vec_dict:
    dist_to_0 = euclidean_dist(weight_vector_tuple, zero_vector, num_weights)
    dist_to_1 = euclidean_dist(weight_vector_tuple, one_vector, num_weights)

    #zero_close_list.append([dist_to_0, weight_vector_tuple])
    #one_close_list.append([dist_to_1, weight_vector_tuple])

    if (len(zero_close_list) < zero_list_len):  # List can grow
      zero_close_list.append([dist_to_0, weight_vector_tuple])
      if (dist_to_0 > max_0_dist):
        max_0_dist = dist_to_0
      zero_close_list.sort()
    elif (dist_to_0 < max_0_dist):  # We have to replace a list element
      zero_close_list = zero_close_list[:-1]  # Remove furthest away w. vector
      zero_close_list.append([dist_to_0, weight_vector_tuple])
      zero_close_list.sort()
      max_0_dist = zero_close_list[-1][0]

    if (len(one_close_list) < one_list_len):  # List can grow
      one_close_list.append([dist_to_1, weight_vector_tuple])
      if (dist_to_1 > max_1_dist):
        max_1_dist = dist_to_1
      one_close_list.sort()
    elif (dist_to_1 < max_1_dist):  # We have to replace a list element
      one_close_list = one_close_list[:-1]  # Remove furthest away w. vector
      one_close_list.append([dist_to_1, weight_vector_tuple])
      one_close_list.sort()
      max_1_dist = one_close_list[-1][0]

  print '  Closest to zero:'
  for (dist, weight_vector_tuple) in zero_close_list:
    print '    With distance to zero of %.4f: %s (%s)' % \
          (dist, weight_vector_to_str(weight_vector_tuple), \
           weight_vec_dict[weight_vector_tuple][1])
  print '  Closest to one:'
  for (dist, weight_vector_tuple) in one_close_list:
    print '    With distance to one of %.4f:  %s (%s)' % \
          (dist, weight_vector_to_str(weight_vector_tuple), \
           weight_vec_dict[weight_vector_tuple][1])
  print

  # Build a dictionary of selected weight vectors
  #
  selected_weight_vec_dict = {}
  for (dist, weight_vector_tuple) in zero_close_list+one_close_list:
    weight_vector_count_match_list = weight_vec_dict[weight_vector_tuple]

    assert weight_vector_tuple not in selected_weight_vec_dict
    selected_weight_vec_dict[weight_vector_tuple] = \
                                                weight_vector_count_match_list

  return selected_weight_vec_dict

# -----------------------------------------------------------------------------
# Function to select weight vectors closest to each of the 2^num_weights
# corners (one weight vector per corner)
#
def select_corners(weight_vec_dict, num_weights):
  """Return selected weight vectors from the given weight vector dictionary
     that are closest to each of the 2^num_weights corners, i.e. one weight
     vector each from [0,..,0], [0,..,1], ... [1,..,1].

     Returns a new dictionary that contains 2^num_weights weight vectors.
  """

  print 'Initial selection of weight vectors closest to corners (one per ' + \
        'corner)'

  # Build a dictionary of selected weight vectors
  #
  selected_weight_vec_dict = {}

  # Get all permutations of 0 / 1 weights
  #
  all_corners_weight_vector_list = [seq for seq in itertools.product([0.0,1.0],
                                    repeat=num_weights)]
  assert len(all_corners_weight_vector_list) == 2**num_weights, \
         (len(all_corners_weight_vector_list), 2**num_weights)

  for corner_weight_vector in all_corners_weight_vector_list:
    min_corner_dist = 99999.0

    for weight_vector_tuple in weight_vec_dict:

      # Only select a weight vector once
      #
      if (weight_vector_tuple not in selected_weight_vec_dict):
        corner_dist = euclidean_dist(weight_vector_tuple, corner_weight_vector,
                                     num_weights)
        if (corner_dist < min_corner_dist):
          min_corner_dist = corner_dist
          min_corner_weight_vector = weight_vector_tuple

    corner_weight_vector_count_match_list = \
              weight_vec_dict[min_corner_weight_vector]
    assert min_corner_weight_vector not in selected_weight_vec_dict
    selected_weight_vec_dict[min_corner_weight_vector] = \
                                          corner_weight_vector_count_match_list
    print '  Corner %s has closest weight vector with distance %.4f: %s (%s)' \
          % (weight_vector_to_str(corner_weight_vector), min_corner_dist,
             weight_vector_to_str(min_corner_weight_vector),
             weight_vec_dict[min_corner_weight_vector][1]) 
  print

  assert len(selected_weight_vec_dict) == 2**num_weights

  return selected_weight_vec_dict

# -----------------------------------------------------------------------------
# Function to select k weight vectors using the farthest first method
#
def select_farthest(weight_vec_dict, k, num_weights):
  """Return k selected weight vectors from the given weight vector dictionary
     based on farthest first approach.

     Returns a new dictionary that contains k weight vectors.
  """

  print 'Farthest first selection of %d weight vectors from %d vectors' % \
        (k, len(weight_vec_dict))

  if (k == len(weight_vec_dict)):  # Return all weight vectors
    selected_weight_vector_dict = weight_vec_dict.copy()
    return selected_weight_vector_dict

  # Keep a dictionary of the so far selected weight vectors
  #
  selected_weight_vector_dict = {}

  # Randomly select the first weight vector
  #
  first_weight_vector = weight_vec_dict.iterkeys().next()

  selected_weight_vector_dict[first_weight_vector] = \
                                          weight_vec_dict[first_weight_vector]

  # Loop until we have selected k+1 weight vectors (then remove the first one)
  #
  while (len(selected_weight_vector_dict) <= k):

    loop_max_dist = -1.0    # The maximum distance of any unselected weight
                            # vector to a selected weight vector
    loop_max_weight_vec = None  # The corresponding record identifier of the
                                # weight vector with this maximum distance

    # Find the weight vector farthest away from all so far selected weight
    # vectors
    #
    for this_weight_vector_tuple in weight_vec_dict:

      # Only consider those not selected so far
      #
      if this_weight_vector_tuple not in selected_weight_vector_dict:

        # Calculate minimum distance of the current weight vector to any so far
        # selected weight vectors
        #
        this_min_dist = 999.0

        for sel_weight_vector_tuple in selected_weight_vector_dict:
          assert sel_weight_vector_tuple != this_weight_vector_tuple

          dist = euclidean_dist(this_weight_vector_tuple, \
                                sel_weight_vector_tuple, num_weights)
          if (dist < this_min_dist):
            this_min_dist = dist

        # Check if this minimum distance is the largest distance in loop so far
        #
        if (this_min_dist > loop_max_dist):
          loop_max_dist =       this_min_dist
          loop_max_weight_vec = this_weight_vector_tuple

    # Add this new farthest weight vector to the selected weight vectors
    #
    selected_weight_vector_dict[loop_max_weight_vec] = \
                                           weight_vec_dict[loop_max_weight_vec]

  del selected_weight_vector_dict[first_weight_vector]

  assert len(selected_weight_vector_dict) == k

  print '  The selected farthest weight vectors are:'
  for sel_weight_vector in selected_weight_vector_dict:
    print '    %s (%s)' % (weight_vector_to_str(sel_weight_vector),
          selected_weight_vector_dict[sel_weight_vector][1])
  print

  return selected_weight_vector_dict

# -----------------------------------------------------------------------------
# Function to select k weight vectors that are densest to other weight vectors
#
def select_densest(weight_vec_dict, k, num_weights):
  """Return k selected weight vectors from the given weight vector dictionary
     using a density based approach.

     Returns a new dictionary that contains k weight vectors.
  """

# TOO slow - n^2 approach - how to make faster? numpy
# Density based clsutering?

  print 'Density-based selection of %d weight vectors from %d vectors' % \
        (k, len(weight_vec_dict))

  # Keep a list with average distance to all other weight vectors for each
  # weight vector
  #
  dist_list = []  # To have tuples (average distance, weight_vector)

  weight_vector_list = weight_vec_dict.keys()

  for this_weight_vector_tuple_1 in weight_vector_list:

    dist_sum = 0.0
    for this_weight_vector_tuple_2 in weight_vector_list:

      if (this_weight_vector_tuple_1 != this_weight_vector_tuple_2):
        dist = euclidean_dist(this_weight_vector_tuple_1, \
                              this_weight_vector_tuple_2, num_weights)
        dist_sum += dist
    # No need to get average as all sums over same number of number of vectors
    #
    dist_list.append((dist_sum, this_weight_vector_tuple_1))

  dist_list.sort()  # Smallest first

  # A dictionary of the so far selected weight vectors
  #
  selected_weight_vector_dict = {}

  # k first elements with smallest distance sums
  #
  for (dist_sum, weight_vector_tuple) in dist_list[:k]:
    selected_weight_vector_dict[weight_vector_tuple] = \
                                         weight_vec_dict[weight_vector_tuple]

  print '  The selected "densest" weight vectors are:'
  for sel_weight_vector in selected_weight_vector_dict:
    print '    %s (%s)' % (weight_vector_to_str(sel_weight_vector),
          selected_weight_vector_dict[sel_weight_vector][1])
  print

  return selected_weight_vector_dict

# -----------------------------------------------------------------------------
# Function to select k weight vectors based on agglomerative clustering (get
# k clusters, then select most central weight vector in each cluster)
#
def select_agglomerative(weight_vec_dict, k, num_weights):
  """Return k selected weight vectors from the given weight vector dictionary
     using an agglomerative clustering approach.

     If the weight vector dictionary is too large then sampling of weight
     vectors is applied.

     Returns a new dictionary that contains k weight vectors.
  """

  num_weight_vec = len(weight_vec_dict)

  print 'Agglomerative clustering based selection of' + \
        ' %d weight vectors from %d vectors' % (k, num_weight_vec)

  weight_vector_tuple_list = weight_vec_dict.keys()

  if (num_weight_vec > k*NUM_SAMPLE):
    num_weight_vec = k*NUM_SAMPLE
    print '  Randomly select %d weight vectors for clustering' % \
          (num_weight_vec)
    random.shuffle(weight_vector_tuple_list)
    weight_vector_tuple_list = weight_vector_tuple_list[:num_weight_vec]

  # Prepare the data for clustering
  #
  cluster_data = numpy.zeros([num_weight_vec, num_weights])

  i = 0
  for weight_vector_tuple in weight_vector_tuple_list:
    cluster_data[:][i] = weight_vector_tuple
    i += 1

  aggl_cluster =   sklearn.cluster.Ward(n_clusters=k, compute_full_tree=False)
  cluster_labels = aggl_cluster.fit_predict(cluster_data)

  assert max(cluster_labels) == (k-1), (max(cluster_labels), k)

  cluster_centroid_list = []

  for i in range(k):
    cluster_centroid_list.append(numpy.zeros(num_weights))

  cluster_size_list = [0]*k

  for i in range(num_weight_vec):
    cluster_num = cluster_labels[i]
    cluster_size_list[cluster_num] += 1
#    this_weight_vector = cluster_data[:][i]

    for j in range(num_weights):
      cluster_centroid_list[cluster_num][j] += cluster_data[i][j]

  assert sum(cluster_size_list) == num_weight_vec
  print '  Cluster sizes:', cluster_size_list

  # Normalise cluster centroids
  #
  for i in range(k):
    for j in range(num_weights):
      cluster_centroid_list[i][j] /= cluster_size_list[i]

  for i in range(k):  # Check all cluster centroids contain normalised values
    assert max(cluster_centroid_list[i]) <= 1.0
    assert min(cluster_centroid_list[i]) >= 0.0

  # Find weight vector closest to each cluster centroid
  #
  centroid_closest_weight_vector_dict = {}  # One per cluster

  for weight_vector_tuple in weight_vec_dict:  # Loop over all weight vectors

    for i in range(k):
      cluster_centroid = cluster_centroid_list[i]
      cluster_min_dist, closest_weight_vector = \
                        centroid_closest_weight_vector_dict.get(i, [99999, []])
      this_dist = euclidean_dist(weight_vector_tuple, cluster_centroid, \
                                 num_weights)
      if (this_dist < cluster_min_dist):
        centroid_closest_weight_vector_dict[i] = (this_dist, \
                                                  weight_vector_tuple)

  # A dictionary of the selected weight vectors
  #
  selected_weight_vector_dict = {}

  for (cluster_min_dist, closest_weight_vector) in \
                             centroid_closest_weight_vector_dict.itervalues():
    selected_weight_vector_dict[closest_weight_vector] = \
                                         weight_vec_dict[closest_weight_vector]

  print '  The selected weight vectors are:'
  for sel_weight_vector in selected_weight_vector_dict:
    print '    %s (%s)' % (weight_vector_to_str(sel_weight_vector),
          selected_weight_vector_dict[sel_weight_vector][1])
  print

  return selected_weight_vector_dict

# -----------------------------------------------------------------------------
# Function to select k weight vectors randomly
#
def select_random(weight_vec_dict, k):
  """Return k randomly selected weight vectors from the given weight vector
     dictionary.

     Returns a new dictionary that contains k weight vectors.
  """

  print 'Random selection of %d weight vectors from %d vectors' % \
        (k, len(weight_vec_dict))

  sel_weight_vec_list = random.sample(weight_vec_dict.keys(), k)

  # A dictionary of the so far selected weight vectors
  #
  selected_weight_vector_dict = {}

  for weight_vec_tuple in sel_weight_vec_list:
    selected_weight_vector_dict[weight_vec_tuple] = \
                                             weight_vec_dict[weight_vec_tuple]

  assert len(selected_weight_vector_dict) == k

  print '  The randomly selected weight vectors are:'
  for sel_weight_vector in selected_weight_vector_dict:
    print '    %s (%s)' % (weight_vector_to_str(sel_weight_vector),
          selected_weight_vector_dict[sel_weight_vector][1])
  print

  return selected_weight_vector_dict

# -----------------------------------------------------------------------------
# Function to find and return the most central weight vectors
#
def select_medoid(weight_vec_dict, k, num_weights):
  """Return the k most central weight vectors as a dictionary, calculated as
     those closest to the centroid.
  """

  print 'Select %d medoids from %d weight vectors' % (k, len(weight_vec_dict))

  if (k == len(weight_vec_dict)):  # Return all weight vectors
    selected_weight_vector_dict = weight_vec_dict.copy()
    return selected_weight_vector_dict

  num_weight_vec = len(weight_vec_dict)

  # First calculate the centroid of the given set of weight vectors
  #
  centroid = [0.0]*num_weights

  for weight_vector_tuple in weight_vec_dict:
    for w in range(num_weights):
      centroid[w] += weight_vector_tuple[w]

  for w in range(num_weights):  # Calculate averages
    centroid[w] /= num_weight_vec

  print '  Centroid weight vector:', weight_vector_to_str(centroid)

  medoid_list = []
  max_medoid_dist = -1.0  # Maximum distance of any medoid to the centroid

  for weight_vector_tuple in weight_vec_dict:
    centroid_dist = euclidean_dist(centroid, weight_vector_tuple, num_weights)

    if (len(medoid_list) < k):  # List can grow
      medoid_list.append([centroid_dist, weight_vector_tuple])
      if (centroid_dist > max_medoid_dist):
        max_medoid_dist = centroid_dist
      medoid_list.sort()
    elif (centroid_dist < max_medoid_dist): # We have to replace a list element
      medoid_list = medoid_list[:-1]  # Remove furthest away weight vector
      medoid_list.append([centroid_dist, weight_vector_tuple])
      medoid_list.sort()
      max_medoid_dist = medoid_list[-1][0]

  medoid_dict = {}

  print '  %d closest weight vectors:' % (k)
  for (dist, weight_vector_tuple) in medoid_list:
    medoid_dict[weight_vector_tuple] = weight_vec_dict[weight_vector_tuple]
    print '    With distance of %.4f: %s (%s)' % \
          (dist, weight_vector_to_str(weight_vector_tuple), \
           weight_vec_dict[weight_vector_tuple][1])
  print

  assert len(medoid_dict) == k

  return medoid_dict

# -----------------------------------------------------------------------------
# Function to perform the oracle
# 
def oracle(weight_vec_dict, acc):
  """Assume a classification (manually) of the given accuracy on the given
     weight vectors.

     The function returns two dictionaries, one of the classified matches and
     one of the classified non matches, the `purity' of classification as the
     maximum of:
      - percentage of weight vectors that are classified as matches,
      - percentage of weight vectors that are classified as non-matches,
     and the entropy of the classification calculated as:

     entropy = - {(M/(M+NM)) * log2(M/(M+NM)) + (NM/(M+NM)) * log2(NM/(M+NM))}

     where M is the number of matches and NM the number of non-matches.

     Purity is a value between 0.5 and 1.0.

     'acc' of these will be correct (i.e. according to their true match status,
     while 1.0 - 'acc' will be wrong.
  """

  print 'Perform oracle with %.2f accuracy on %d weight vectors' % \
        (100.0*acc, len(weight_vec_dict))

  match_dict =     {}
  non_match_dict = {}

  num_weight_vec = len(weight_vec_dict)

  num_correct = int(round(acc*num_weight_vec))
  print '  The oracle will correctly classify %d weight vectors and ' % \
        (num_correct) + 'wrongly classify %d' % (num_weight_vec-num_correct)

  weight_vector_list = weight_vec_dict.keys()
  random.shuffle(weight_vector_list)

  # Make sure the weight vectors are unique
  #
  assert len(set(weight_vector_list)) == len(weight_vector_list)

  # Get the list of weight vectors to be classified correctly and wrongly
  #
  corr_list = random.sample(weight_vector_list, num_correct)
  wrong_list = list(set(weight_vector_list) - set(corr_list))
  assert len(set(corr_list).intersection(set(wrong_list))) == 0

  num_tp = 0
  num_fp = 0
  num_tn = 0
  num_fn = 0

  # Perform the oracle classification
  #
  for weight_vector_tuple in corr_list:  # The correct classification
    weight_vector_count_match_list = weight_vec_dict[weight_vector_tuple]
    if (weight_vector_count_match_list[1] == True):  # A true match
      match_dict[weight_vector_tuple] = weight_vector_count_match_list
      num_tp += 1
    else:
      non_match_dict[weight_vector_tuple] = weight_vector_count_match_list
      num_tn += 1

  for weight_vector_tuple in wrong_list:  # The wrong classification
    weight_vector_count_match_list = weight_vec_dict[weight_vector_tuple]
    if (weight_vector_count_match_list[1] == True):  # A true match
      non_match_dict[weight_vector_tuple] = weight_vector_count_match_list
      num_fn += 1
    else:
      match_dict[weight_vector_tuple] = weight_vector_count_match_list
      num_fp += 1

  m =  float(len(match_dict))       # Number of matches
  nm = float(len(non_match_dict))   # Number of non-matches
  a =  float(len(weight_vec_dict))  # Number of all

  purity = max(m/a, nm/a)
  assert purity >= 0.5 and purity <= 1.0, purity

  if (m != 0.0) and (nm != 0.0):
    entropy = - m/a * math.log(m/a, 2) - nm/a * math.log(nm/a, 2)
  else:
    entropy = 0.0

  assert entropy >= 0.0 and entropy <= 1.0, entropy

  print '  Classified %d matches and %d non-matches' % \
        (len(match_dict), len(non_match_dict))
  print '    Purity of oracle classification:  %.3f' % (purity)
  print '    Entropy of oracle classification: %.3f' % (entropy)
  print '    Number of true matches:     ', num_tp
  print '    Number of false matches:    ', num_fp
  print '    Number of true non-matches: ', num_tn
  print '    Number of false non-matches:', num_fn
  print
  assert num_tp+num_fp == len(match_dict)
  assert num_tn+num_fn == len(non_match_dict)

  if (len(match_dict) == 0):
    print '*** Warning: Oracle returns an empty match dictionary ***'
  if (len(non_match_dict) == 0):
    print '*** Warning: Oracle returns an empty non-match dictionary ***'

  return match_dict, non_match_dict, purity, entropy

# -----------------------------------------------------------------------------
# Function to perform k nearest neighbour classification and split the cluster
#
def knn_split_classifier(weight_vector_dict, k, match_dict, non_match_dict):
  """Split the given weight vector dictionary into matches and non-matches
     according to the match and non-match training dictionaries given using
     a k nearest neighbour classifier.

     Returns two new weight vector dictionaries, one for the classified matches
     and the other for the classified non-matches.
  """

  assert k%2 == 1, k  # k must be odd

  print '%d-NN classification of %d weight vectors' % \
        (k, len(weight_vector_dict))
  print '  Based on %d matches and %d non-matches' % \
        (len(match_dict), len(non_match_dict))

  # The two dictionaries of classified weight vectors to be generated
  #
  match_weight_vector_dict =     {}
  non_match_weight_vector_dict = {}

  for (weight_vector_tuple, weight_vector_count_match_list) in \
                                              weight_vector_dict.iteritems():

    this_dist_list = []  # List of this weight vector's distances to all
                         # training vectors

    for train_weight_vector in match_dict.iterkeys():
      dist = euclidean_dist(weight_vector_tuple, train_weight_vector,
                            num_weights)
      this_dist_list.append((dist, 'm'))  # Distance to a match

    for train_weight_vector in non_match_dict.iterkeys():
      dist = euclidean_dist(weight_vector_tuple, train_weight_vector,
                            num_weights)
      this_dist_list.append((dist, 'nm'))  # Distance to a non-match

    this_dist_list.sort()
    this_dist_list_k = this_dist_list[:k]  # k closest training vectors

    num_matches, num_non_matches = 0, 0

    for (dist, match_status) in this_dist_list_k:
      if (match_status == 'm'):
        num_matches += 1
      else:
        num_non_matches += 1

    if (num_matches > num_non_matches):
      match_weight_vector_dict[weight_vector_tuple] = \
                                                 weight_vector_count_match_list
    else:
      non_match_weight_vector_dict[weight_vector_tuple] = \
                                                 weight_vector_count_match_list

  assert len(match_weight_vector_dict) + len(non_match_weight_vector_dict) == \
         len(weight_vector_dict)

  print '  Classified %d matches and %d non-matches' % \
        (len(match_weight_vector_dict), len(non_match_weight_vector_dict))
  print

  return match_weight_vector_dict, non_match_weight_vector_dict

# -----------------------------------------------------------------------------
# Function to perform SVM classification and split the cluster
#
def svm_split_classifier(weight_vector_dict, match_dict, non_match_dict,
                         num_weights):
  """Split the given weight vector dictionary into matches and non-matches
     according to the match and non-match training dictionaries given using an
     SVM classifier.

     Returns two new weight vector dictionaries, one for the classified matches
     and the other for the classified non-matches.
  """

  print 'SVM classification of %d weight vectors' % (len(weight_vector_dict))
  print '  Based on %d matches and %d non-matches' % \
        (len(match_dict), len(non_match_dict))

  # Prepare the training and test data sets for the classifier
  #
  num_train_vec = len(match_dict) + len(non_match_dict)
  num_test_vec =  len(weight_vector_dict)

  train_data =    numpy.zeros([num_train_vec, num_weights])
  train_class =   numpy.zeros(num_train_vec)
  train_weights = numpy.zeros(num_train_vec)

  test_data =  numpy.zeros([num_test_vec, num_weights])
  test_class = numpy.zeros(num_test_vec)

  j = 0
  for (weight_vector_tuple, weight_vector_count_match_list) in \
                                   match_dict.iteritems():
    train_data[:][j] = weight_vector_tuple
    train_class[j] =   1.0
    train_weights[j] = weight_vector_count_match_list[0]
    assert train_weights[j] >= 1
    j += 1
  for (weight_vector_tuple, weight_vector_count_match_list) in \
                                non_match_dict.iteritems():
    train_data[:][j] = weight_vector_tuple
    train_class[j] =   0.0
    train_weights[j] = weight_vector_count_match_list[0]
    assert train_weights[j] >= 1
    j += 1

  weight_vectors_to_classify_list = weight_vector_dict.items()

  j = 0
  for (weight_vector_tuple, weight_vector_count_match_list) in \
                                   weight_vectors_to_classify_list:
    test_data[:][j] = weight_vector_tuple
    match_status = weighted_unique_weight_vec_dict[weight_vector_tuple][1]
    assert match_status in [True, False]
    if (match_status == True):
      test_class[j] = 1.0
    else:
      test_class[j] = 0.0
    j += 1

  classifier = sklearn.svm.SVC(kernel='linear', C=0.1)
  classifier.fit(train_data, train_class, sample_weight=train_weights)
  class_predict = classifier.predict(test_data)

  # The two dictionaries of classified weight vectors to be generated
  #
  match_weight_vector_dict =     {}
  non_match_weight_vector_dict = {}

  num_matches, num_non_matches = 0, 0

  for i in range(num_test_vec):
    weight_vector_tuple, weight_vector_count_match_list = \
                                            weight_vectors_to_classify_list[i]
    if (class_predict[i] == 1.0):
      num_matches += 1
      match_weight_vector_dict[weight_vector_tuple] = \
                                                weight_vector_count_match_list
    else:
      num_non_matches += 1
      non_match_weight_vector_dict[weight_vector_tuple] = \
                                                weight_vector_count_match_list

  assert len(match_weight_vector_dict) + len(non_match_weight_vector_dict) == \
         len(weight_vector_dict)

  print '  Classified %d matches and %d non-matches' % \
        (len(match_weight_vector_dict), len(non_match_weight_vector_dict))
  print

  return match_weight_vector_dict, non_match_weight_vector_dict

# -----------------------------------------------------------------------------
# Function to perform decision tree classification and split the cluster
#
def dtree_split_classifier(weight_vector_dict, match_dict, non_match_dict,
                           num_weights):
  """Split the given weight vector dictionary into matches and non-matches
     according to the match and non-match training dictionaries given using a
     decision tree classifier.

     Returns two new weight vector dictionaries, one for the classified matches
     and the other for the classified non-matches.
  """

  print 'Decision tree classification of %d weight vectors' % \
        (len(weight_vector_dict))
  print '  Based on %d matches and %d non-matches' % \
        (len(match_dict), len(non_match_dict))

  # Prepare the training and test data sets for the classifier
  #
  num_train_vec = len(match_dict) + len(non_match_dict)
  num_test_vec =  len(weight_vector_dict)

  train_data =    numpy.zeros([num_train_vec, num_weights])
  train_class =   numpy.zeros(num_train_vec)
  train_weights = numpy.zeros(num_train_vec)

  test_data =  numpy.zeros([num_test_vec, num_weights])
  test_class = numpy.zeros(num_test_vec)

  j = 0
  for (weight_vector_tuple, weight_vector_count_match_list) in \
                                   match_dict.iteritems():
    train_data[:][j] = weight_vector_tuple
    train_class[j] =   1.0
    train_weights[j] = weight_vector_count_match_list[0]
    assert train_weights[j] >= 1
    j += 1
  for (weight_vector_tuple, weight_vector_count_match_list) in \
                                non_match_dict.iteritems():
    train_data[:][j] = weight_vector_tuple
    train_class[j] =   0.0
    train_weights[j] = weight_vector_count_match_list[0]
    assert train_weights[j] >= 1
    j += 1

  weight_vectors_to_classify_list = weight_vector_dict.items()

  j = 0
  for (weight_vector_tuple, weight_vector_count_match_list) in \
                                   weight_vectors_to_classify_list:
    test_data[:][j] = weight_vector_tuple
    match_status = weighted_unique_weight_vec_dict[weight_vector_tuple][1]
    assert match_status in [True, False]
    if (match_status == True):
      test_class[j] = 1.0
    else:
      test_class[j] = 0.0
    j += 1

  classifier = sklearn.tree.DecisionTreeClassifier(criterion='gini')
  classifier.fit(train_data, train_class, sample_weight=train_weights)
  class_predict = classifier.predict(test_data)

  # The two dictionaries of classified weight vectors to be generated
  #
  match_weight_vector_dict =     {}
  non_match_weight_vector_dict = {}

  num_matches, num_non_matches = 0, 0

  for i in range(num_test_vec):
    weight_vector_tuple, weight_vector_count_match_list = \
                                            weight_vectors_to_classify_list[i]
    if (class_predict[i] == 1.0):
      num_matches += 1
      match_weight_vector_dict[weight_vector_tuple] = \
                                                weight_vector_count_match_list
    else:
      num_non_matches += 1
      non_match_weight_vector_dict[weight_vector_tuple] = \
                                                weight_vector_count_match_list

  assert len(match_weight_vector_dict) + len(non_match_weight_vector_dict) == \
         len(weight_vector_dict)

  print '  Classified %d matches and %d non-matches' % \
        (len(match_weight_vector_dict), len(non_match_weight_vector_dict))
  print

  return match_weight_vector_dict, non_match_weight_vector_dict

# -----------------------------------------------------------------------------
# Function to calculate distance of a cluster from 0 corner
#
def cluster_0_dist(weight_vector_dict, num_weights, dist_mode):
  """Calculate distance of a cluster from the [0] corner using one of minimum,
     average or maximum distance (corresponding to single, average or complete
     link), based on Euclidean distance.

     num_weights gives the dimensionality of the weight vectors.

     Returns a numerical distance value.
  """

  assert dist_mode in ['single', 'average', 'complete'], dist_mode

  zero_vec = [0]*num_weights

  min_dist = 999999.99

  if (dist_mode == 'single'):
    for weight_vector_tuple in weight_vector_dict:
      print weight_vector_tuple
      this_dist = euclidean_dist(zero_vec, weight_vector_tuple, num_weights)
      if (this_dist < min_dist):
        min_dist = this_dist

  elif (dist_mode == 'complete'):
    for weight_vector_tuple in weight_vector_dict:
      print weight_vector_tuple
      this_dist = euclidean_dist(zero_vec, weight_vector_tuple, num_weights)
      if (this_dist > min_dist):
        min_dist = this_dist

  else:  # Calculate average distance
    num_weight_vec = len(weight_vector_dict)
    avrg_dist_vec = [0.0]*num_weights
    for weight_vector_tuple in weight_vector_dict:
      for i in range(num_weights):
        avrg_dist_vec[i] += weight_vector_tuple[i]
    for i in range(num_weights):
      avrg_dist_vec[i] /= num_weight_vec
    min_dist = euclidean_dist(zero_vec, avrg_dist_vec, num_weights)

  assert min_dist >= 0, min_dist

  return min_dist

# -----------------------------------------------------------------------------
# Function to calculate number of samples needed from a cluster to obtain
# enough examples for a given confidence and margin of error
#
def get_sample_size(cluster_size, est_proportion, sample_error):
  """Follows equation on page 505 of An Introduction to Statistical Methods
     and Data analysis, Ott and Longnecker, 6th edition.

     We assume a confidence level of 95%.
  """

  z_alpha2 = 1.95  # For 95% confidence interval

  sample_size = z_alpha2**2 * est_proportion * (1.0 - est_proportion) \
                / (sample_error**2)
  #print 'sample size:', sample_size,cluster_size,est_proportion,sample_error

  # For small cluster sizes we adjust:
  #
  sample_size_adj = cluster_size*sample_size / (cluster_size + sample_size)

  #print 'Sample size:', int(math.ceil(sample_size_adj))

  #if (sample_size != sample_size_adj):
  #  print '  Original sample size: %.2f and adjusted sample size: %.2f' % \
  #        (sample_size, sample_size_adj)
  #print

  return int(math.ceil(sample_size_adj))

def atualizaDM_NDM(dic, file1, file2):

    dmfile  = open(file1, 'ab')
    ndmfile = open(file2, 'ab')

    writer_dm = csv.writer(dmfile, delimiter=';', quotechar = ' ')
    writer_ndm = csv.writer(ndmfile, delimiter=';', quotechar = ' ')


    for pesos in oracle_class_cache_set:

        for k,v in dic.iteritems(): 
            if pesos == v[1]:

                ids = k.split('~')

                if v[0]:
                    print 'Verdadeiro!'
                    writer_dm.writerow(ids)
                else:
                    print 'Falso!'
                    writer_ndm.writerow(ids)

                break

    dmfile.close()
    ndmfile.close()
    


#Geração do conjunto de treinamento
def geraTrainSet(dic, dir, file1):
    
    try:
        os.makedirs(dir)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise

    #cont = 0
    
#     trainSet  = open(dir+file1, 'ab')
    trainSet  = open(dir+file1, 'wb')

    writer_ts = csv.writer(trainSet, delimiter=';', quotechar = ' ')
 
    for pesos in oracle_class_cache_set:

        for k,v in dic.iteritems(): 
            if pesos == v[1]:
                ids = k.split('~')

                if v[0]:
                    #print 'Verdadeiro!'
                    #lista = [cont] + ids + list(v[1]) + [1.0]
                    lista = list(v[1]) + [1.0]
                    writer_ts.writerow(lista)
                else:
                    #print 'Falso!'
                    #lista = [cont] + ids + list(v[1]) + [0.0]
                    lista = list(v[1]) + [0.0]
                    writer_ts.writerow(lista)
                #cont = cont + 1
                break
        
    trainSet.close()
	
#Geração do conjunto de treinamento
def geraTestSet(dic, dir, file1):
    
    try:
        os.makedirs(dir)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise

    #cont = 0
    cont1 = 0
    
#     testSet  = open(dir+file1, 'ab')
    testSet  = open(dir+file1, 'wb')

    writer_ts = csv.writer(testSet, delimiter=';', quotechar = ' ')
 
	#Remoção dos vetores selecionados para o conjunto de treinamento
    for pesos in oracle_class_cache_set:

        for k,v in dic.iteritems(): 
            if pesos == v[1]:
                
                del dic[k]
                cont1 = cont1 + 1
                break
	
	#print 'Número de deleções: %d' %(cont1)
	
	#Geração do conjunto de teste
    for k,v in dic.iteritems(): 
            
        ids = k.split('~')

        if v[0]:
            #lista = [cont] + ids + list(v[1]) + [1.0]
            lista = list(v[1]) + [1.0]
            writer_ts.writerow(lista)
        else:
            #lista = [cont] + ids + list(v[1]) + [0.0]
            lista = list(v[1]) + [0.0]
            writer_ts.writerow(lista)
        #cont = cont + 1
        
        
    testSet.close()
    
# def atualizaEstatAA(permutacao, dir):
    
#     With open(dir, rb) as f:
#     reader = csv.reader(f)
#     for row in reader:
#         if row[2] = permutacao #Coluna da permutação
#             print
#             print row

# import pandas as pd

#Acho que não vai ser uma função.
#Melhor criar o dataframe antes da primeira iteração ea atualizá-lo ao término de cada uma
#Depois disso fecha o arquivo e salva
# def atualizaEstatAA(permutacao, arq):
    
#     estatisticas = pd.read_csv(arq)
    
#     estatisticas.set_index('permutacao')
        
#         linha = estatisticas.loc[permutacao, : ] #Armazena a linha correspondente à permutação
        
#         tp = linha[: tp] + tp 
        #Idem para fp, tn e fn
        #calcula as métricas além de armazenar os dados de inspecoesManuais, dm, ndm e permutacao e a etapa "AA"
        
        #Armazena no final do arquivo
        
        


# def setVetorSim(nome):
#     print 'Entrei em setVetorSim() com o arquivo %s' %(nome)
#     weight_vec_file_name = nome
#     print 'weight_vec_file_name = %s' %(weight_vec_file_name)
    
# def getNomeVetorSim():
#     return weight_vec_file_name

# =============================================================================
# Main program

# Step 1: Load, analyse and clean the weight vectors file
#

#Repetições para os experimentos

bases = ["DBLP2-ACM"] #Só funciona para uma base, por que tem-se que alterar os dados iniciais referentes às colunas
# qps = ["QP1", "QP2"] #Todos menos QP6
# qps = ["qp1", "qp2b", "qp2m", "qp2r", "qp3all", "qp3lot"] #Todos menos QP6 e QP7
qps = ["qp1"]

for base in bases:
    
    print("Base atual: {0}".format(base))
    
    for qp in qps:
        
        print("QP atual: {0}".format(qp))
        
        dirOrig = "../csv/conjuntosDS/conjuntosDivergAA/"+base+"/"+qp+"/"
        dirEstat = "../csv/estatisticas/"+base+"/"+qp+"/"
        estat = dirEstat+"estatisticaDS.csv"

#         dirOrig = "../csv/conjuntosDS/conjuntosDiverg/"
#         estat = "../csv/estatisticaDS.csv"

        # Diretórios para Windows
        # dirOrig = "C:\Users\Diego\Documents\NetBeansProjects\Master-SKYAM\AS\src\csv\conjuntosDS\conjuntosDiverg\\"
        # estat = "C:\Users\Diego\Documents\NetBeansProjects\Master-SKYAM\AS\src\csv\estatisticaDS.csv"


        # dirOrig = "..\..\Documents\NetBeansProjects\Master-SKYAM\AS\src\csv\conjuntosDS\conjuntosDiverg"
        # estat = "..\..\Documents\NetBeansProjects\Master-SKYAM\AS\src\csv\estatisticaDS.csv"

        # Diretórios para Linux
        # dirOrig = "./arqResult/csv/conjuntosDS/conjuntosDiverg/"
        # estat = "./arqResult/csv/estatisticaDS.csv"

        # /home/diego/anaconda3/rerequestingofalgorithmsforconductingresearch/arqResult/csv

        # /home/diego/anaconda3/rerequestingofalgorithmsforconductingresearch/arqResult/csv

        etapa = '2 - AA[pet-chr]'

        import pandas as pd
        import re

        estatisticas = pd.read_csv(estat, index_col=['algoritmosUtilizados', 'etapa', 'permutacao'], sep=';')
        # estatisticas.set_index('permutacao')
        estatisticas.head()
        print(estatisticas.columns)


        print 'estatisticas.shape'
        print estatisticas.shape

        arquivos = [] #Adicionado depois

        for _, _, arquivo in os.walk(dirOrig):
             #print(arquivo)
             arquivos.extend(arquivo)   

        #print 'quantidade de arquivos: %d' %(len(arquivos))        

        #linhaAtual
        #cont = 0

        #for arq in arquivo:
        for arq in arquivos:
            #print 'Diego %d' %(cont)

            if '_NEW' in arq:
#             if ('diverg(10)1_NEW' in arq): #Aqui, Diego!
                print 'Analisando o arquivo: %s' %(arq)
                #cont+=1
        #         num = arq.replace('diverg','')
        #         num = num.replace('_NEW.csv','')
        #'''

                num = re.sub('diverg.*\)', r'', arq) #Alterar para fazer a substituição de tudo em uma linha só
                num = num.replace('_NEW.csv','')
        #         print(num)

                algUtl = re.sub('diverg.*\(', r'', arq) #Alterar para fazer a substituição de tudo em uma linha só
                algUtl = re.sub('\).*', r'', algUtl) #Alterar para fazer a substituição de tudo em uma linha só
        #         alg = alg.replace('\).*','')
        #         print 'alg: %s' %(algUtl)
                algUtl = int(algUtl)
                #Passando o csv para selecionar os vetores para conjunto de treinamento
        #         setVetorSim(arq)

        #         print 'Está sendo analisado o arquivo %s' %(getNomeVetorSim())

        #       
                permutacao = int(num)


        #         linhaAtual = estatisticas.xs((algUtl, '1 - acm diverg', permutacao))
                linhaAtual = estatisticas.xs((algUtl, '1 - acm diverg', permutacao))
                #linhaAtual = estatisticas.loc[algUtl, '1 - acm diverg', permutacao, : ] #Armazena a linha correspondente à permutação
        #         linhaAtual = estatisticas.loc[['1 - acm diverg', permutacao], : ] #Armazena a linha correspondente à permutação

        #         linhaAtual.tolist()
#                 print type(linhaAtual)
#                 print 'Linha atual aqui, jovem!'
#                 print linhaAtual.shape
#                 print linhaAtual
        #         print 'colunas'
        #         print linhaAtual.columns
        #         print linhaAtual
                #print type(linhaAtual['precision'])#[:] #.get_value
                #print linhaAtual['precision'].item()#[:] #.get_value

        #         linha




                start_time = time.time()

        #         weights_name_list, weight_vector_dict = \
        #                         load_weight_vec_file(weight_vec_file_name, weights_to_use_list)
                weights_name_list, weight_vector_dict = \
                                load_weight_vec_file(dirOrig+arq, weights_to_use_list)
                file_num_weight_vectors = len(weight_vector_dict)

                  #Adicionado por Diego

                #weight_vector_dict_orig = copy.deepcopy(weight_vector_dict)
                weight_vector_dict_orig = dict(weight_vector_dict)

                weight_vector_dict, weighted_unique_weight_vec_dict, num_true_matches, \
                       num_true_non_matches = analyse_filter_weight_vectors(weight_vector_dict)

                unique_num_weight_vectors = len(weighted_unique_weight_vec_dict)

                data_prep_time = time.time() - start_time
                print 'Time to load and analyse the weight vector file: %.2f sec' % \
                     (data_prep_time)
                print

                # A flag, set to True once the first (initial) selection has been done (as the
                # initial selection function is different from all following ones)
                #
                init_selection_done = False

                cluster_queue = []  # Clusters of weight vectors that need to be split further

                cluster_size_list =     []  # Collect statistics about cluster sizes, their
                cluster_pureness_list = []  # pureness, entropy and time required
                cluster_entropy_list =  []
                cluster_use_pure_list = []  # Purness of only clusters used for training
                cluster_sample_size =   []  # Number of samples required per cluster
                loop_sel_time_list =    []
                loop_oracle_time_list = []
                loop_class_time_list =  []
                num_clusters_used =      0  # How many clusters were used for training

                # Calculate initial estimated proportion of matches as minimum data set size
                # divided by number of weight vectors
                #
                cluster_est_proportion = float(min_data_set_size) / unique_num_weight_vectors

                cluster_est_proportion = 0.5  # Gives largest possible sample

                print 'Initial estimated match proportion: %.3f' % \
                      (cluster_est_proportion)
                print

                # The queue contains tuples with:
                # (weight vector dictionary, cluster purity, cluster_entropy, cluster size,
                # cluster_est_proportion)

                # Start with all weight vectors in one cluster

                # For the initial cluster we set purity to 0.5 (minimum possible value) as it
                # is unknown; similar maximum entropy is 1.0
                #
                cluster_queue.append((weighted_unique_weight_vec_dict, 0.5, 1.0,
                                      len(weighted_unique_weight_vec_dict),
                                      cluster_est_proportion))

                # Keep a set of all weight vectors 'manually' classified by the oracle (so we
                # can check the budget and stop once maximum budget is reached)
                #
                oracle_class_cache_set = set()

                # Two dictionaries with the weight vectors selected as training data
                #
                final_match_weight_vector_dict =     {}
                final_non_match_weight_vector_dict = {}

                # Step 2: Recursively split clusters until stopping criteria achieved
                #
                loop_count = 0

                while (cluster_queue != []):  # As long as we have clusters to be split further
                  loop_count += 1

                  print '- '*40
                  print 'Loop %d: Queue length: %d' % (loop_count, len(cluster_queue))
                  print '  Number of manual oracle classifications performed:', \
                        len(oracle_class_cache_set)
                  print '  Size, purity, entropy, and estimated match proportion of ' + \
                        'clusters in queue:'
                  for cluster_tuple in cluster_queue:
                    print '   ', (cluster_tuple[3], cluster_tuple[1], cluster_tuple[2], \
                                  cluster_tuple[4])
                  print
                  print 'Current size of match and non-match training data sets: %d / %d' % \
                        (len(final_match_weight_vector_dict),
                         len(final_non_match_weight_vector_dict))
                  print

                  # Step 2a: Select a cluster from the queue

                  # Extension January 2015: Different orderings of the blocks in the queue
                  # 'fifo' First in first out - our current approach
                  # 'random' Random order - use random.choice()
                  # 'max_puri'  Clusters with highest purity first (purity from parent cluster)
                  # 'min_puri'  Clusters with lowest purity first (purity from parent cluster)
                  # 'max_size'  Largest clusters first
                  # 'min_size'  Smallest clusters first
                  # 'min_entr'  Clusters with lowest entropy first (based on entropy of parent
                  #             cluster, as child cluster entropy can only be smaller)
                  # 'max_entr'  Clusters with highest entropy first (again based on parent
                  #             cluster entropy)
                  # 'max_size'  Largest clusters first
                  # 'min_size'  Smallest clusters first
                  # 'close_01'  Closest to 0 or 1 corner
                  # 'close_mid' Closest to middle (half between 0 and 1 corners)
                  # 'balance'   Select so that training set sizes are balanced
                  # 'sample'    Select cluster which has largest ratio of cluster size divided
                  #             by number of number of samples required

                  # Need to calculate distances of all clusters to 0 corner first
                  #
                  if (queue_order in ['close_01','close_mid','balance']):  

                    cluster_dist_list = []  # Pairs of distances and corresponding clusters

                    for cluster_tuple in cluster_queue:
                      cluster_dist = cluster_0_dist(cluster_tuple[0], num_weights, \
                                                    CLUSTER_MIN_DIST)
                      if (queue_order == 'close_01'):
                        abs_dist = min(cluster_dist, 1.0-cluster_dist)  # Distance from corner
                        cluster_dist_list.append((abs_dist, cluster_tuple))
                      elif (queue_order == 'close_mid'):
                        abs_dist = abs(0.5-cluster_dist)  # Distance from middle
                        cluster_dist_list.append((abs_dist, cluster_tuple))
                      elif (queue_order == 'balance'):
                        cluster_dist_list.append((cluster_dist, cluster_tuple))
                      else:
                        raise Exception, queue_order

                    cluster_dist_list.sort()  # Smallest distance first

                    if (queue_order in ['close_01', 'close_mid']):  # Take first
                      next_cluster_tuple = cluster_dist_list[0][1]
                      cluster_queue.remove(next_cluster_tuple)
                    else:  # Select closest depending on size of current training data sets
                      num_matches =     len(final_match_weight_vector_dict)
                      num_non_matches = len(final_non_match_weight_vector_dict)

                      # Use <= to favour selecting likely matches over non-matches
                      #
                      if (num_matches <= num_non_matches): # Select cluster closest to 1 corner
                        print '  Balanced cluster selection, select cluster closest to' + \
                              ' [1,..,1]'
                        print
                        next_cluster_tuple = cluster_dist_list[-1][1]
                        cluster_queue.remove(next_cluster_tuple)
                      else:
                        print '  Balanced cluster selection, select cluster closest to' + \
                              '[0,...,0]'
                        print
                        next_cluster_tuple = cluster_dist_list[0][1]
                        cluster_queue.remove(next_cluster_tuple)

                  elif (queue_order == 'fifo'):
                    next_cluster_tuple = cluster_queue.pop(0)

                  elif (queue_order == 'random'):
                    next_cluster_tuple = random.choice(cluster_queue)
                    cluster_queue.remove(next_cluster_tuple)

                  elif (queue_order == 'max_puri'):
                    check_max_purity = 0.0
                #    next_cluster_tuple = None
                    for cluster_tuple in cluster_queue:
                      if (cluster_tuple[1] > check_max_purity):
                        check_max_purity = cluster_tuple[1]
                        next_cluster_tuple = cluster_tuple
                    cluster_queue.remove(next_cluster_tuple)

                  elif (queue_order == 'min_puri'):
                    check_min_purity = 2.0
                #    next_cluster_tuple = None
                    for cluster_tuple in cluster_queue:
                      if (cluster_tuple[1] < check_min_purity):
                        check_min_purity = cluster_tuple[1]
                        next_cluster_tuple = cluster_tuple
                    cluster_queue.remove(next_cluster_tuple)

                  elif (queue_order == 'max_entr'):
                    check_max_entropy = -1.0
                #    next_cluster_tuple = None
                    for cluster_tuple in cluster_queue:
                      if (cluster_tuple[2] > check_max_entropy):
                        check_max_entropy = cluster_tuple[2]
                        next_cluster_tuple = cluster_tuple
                    cluster_queue.remove(next_cluster_tuple)

                  elif (queue_order == 'min_entr'):
                    check_min_entropy = 2.0
                #    next_cluster_tuple = None
                    for cluster_tuple in cluster_queue:
                      if (cluster_tuple[2] < check_min_entropy):
                        check_min_entropy = cluster_tuple[2]
                        next_cluster_tuple = cluster_tuple
                    cluster_queue.remove(next_cluster_tuple)

                  elif (queue_order == 'max_size'):
                    check_max_size = -1.0
                #    next_cluster_tuple = None
                    for cluster_tuple in cluster_queue:
                      if (cluster_tuple[3] > check_max_size):
                        check_max_size = cluster_tuple[2]
                        next_cluster_tuple = cluster_tuple
                    cluster_queue.remove(next_cluster_tuple)

                  elif (queue_order == 'min_size'):
                    check_min_size = 99999999.0
                #    next_cluster_tuple = None
                    for cluster_tuple in cluster_queue:
                      if (cluster_tuple[3] < check_min_size):
                        check_min_size = cluster_tuple[2]
                        next_cluster_tuple = cluster_tuple
                    cluster_queue.remove(next_cluster_tuple)

                  elif (queue_order == 'sample'):
                    check_max_ratio = -1.0

                    for cluster_tuple in cluster_queue:
                      this_cluster_size = cluster_tuple[3]
                      this_cluster_prop = cluster_tuple[4]
                      this_select_num = get_sample_size(this_cluster_size,
                                                        this_cluster_prop, sample_error)
                      this_cluster_ratio = float(this_cluster_size) / this_select_num
                      if (this_cluster_ratio > check_max_ratio):
                        check_max_ratio = this_cluster_ratio
                        next_cluster_tuple = cluster_tuple
                    cluster_queue.remove(next_cluster_tuple)

                ## TODO: Add optimal ordering - combine balancing, sample size, purity etc.

                  else:
                    raise Exception, queue_order

                  (next_weight_vec_dict, cluster_purity, cluster_entropy, cluster_size, \
                                                  cluster_est_proportion) = next_cluster_tuple

                  cluster_size_list.append(cluster_size)

                  print 'Selected cluster with (queue ordering: %s):' % (queue_order)
                  print '- Purity %.2f and entropy %.2f' % (cluster_purity, cluster_entropy)
                  print '- Size %d weight vectors' % (cluster_size)
                  print '- Estimated match proportion %.3f' % (cluster_est_proportion)
                  print

                  # Calculate sample size (number of weight vectors to select) for this cluster
                  #
                  select_num = get_sample_size(cluster_size, cluster_est_proportion, \
                                               sample_error)
                  cluster_sample_size.append(select_num)

                  print 'Sample size for this cluster:', select_num
                  print

                  # Step 2b: Get representative weight vectors from this cluster
                  #
                  start_time = time.time()

                  if (init_selection_done == False):  # Do initial selection
                    init_selection_done = True
                    print 'Perform initial selection using "%s" method' % (init_method)
                    print

                    if (init_method == 'far'):
                      rep_weight_vec_dict = select_farthest(next_weight_vec_dict, select_num, \
                                                            num_weights)
                    elif (init_method == '01'):
                      rep_weight_vec_dict = select_01(next_weight_vec_dict, select_num, \
                                                      num_weights)
                    elif (init_method == 'corner'):
                      rep_weight_vec_dict = select_corners(next_weight_vec_dict, num_weights)

                    else:  # Random
                     rep_weight_vec_dict = select_random(next_weight_vec_dict, select_num)

                  else:  # Any following selection
                    if (select_method == 'far'):
                      rep_weight_vec_dict = select_farthest(next_weight_vec_dict, select_num, \
                                                            num_weights)
                    elif (select_method == 'far_med'):
                      rep_weight_vec_dict = select_farthest(next_weight_vec_dict, select_num, \
                                                            num_weights)
                      centroid_weight_vec_dict = select_medoid(next_weight_vec_dict, 1, \
                                                               num_weights)
                      assert len(centroid_weight_vec_dict) == 1

                      # Add centroids to representative weight vectors
                      #
                      for weight_vector_tuple in centroid_weight_vec_dict:
                        rep_weight_vec_dict[weight_vector_tuple] = \
                                                  centroid_weight_vec_dict[weight_vector_tuple]

                    elif (select_method == 'dense'):
                      rep_weight_vec_dict = select_densest(next_weight_vec_dict, select_num, \
                                                           num_weights)

                    elif (select_method == 'aggl'):
                      rep_weight_vec_dict = select_agglomerative(next_weight_vec_dict,
                                                                 select_num, \
                                                                 num_weights)

                    else:  # Random
                      rep_weight_vec_dict = select_random(next_weight_vec_dict, select_num)

                  sel_time = time.time() - start_time
                  loop_sel_time_list.append(sel_time)

                  # Step 2c: Give selected weight vectors to oracle for 'manual' classification
                  #
                  start_time = time.time()

                  match_dict, non_match_dict, purity, entropy = \
                                                       oracle(rep_weight_vec_dict, oracle_acc)
                  cluster_pureness_list.append(purity)
                  cluster_entropy_list.append(entropy)

                  assert len(match_dict) + len(non_match_dict) == len(rep_weight_vec_dict)

                  # Calculate a new estimate for match proportion based on size of manually
                  # classified weight vectors
                  #
                  cluster_est_proportion = float(len(match_dict)) / len(rep_weight_vec_dict)

                  # Update the cache with the 'manually' classified weight vectors
                  #
                  for weight_vector_tuple in rep_weight_vec_dict:
                    oracle_class_cache_set.add(weight_vector_tuple)

                  # Add the manually classified weight vectors into the final training sets
                  # and remove from current cluster as well as from the original weight vector
                  # dictionary
                  #
                  for weight_vector_tuple in match_dict:
                    final_match_weight_vector_dict[weight_vector_tuple] = \
                                                                match_dict[weight_vector_tuple]
                    del next_weight_vec_dict[weight_vector_tuple]
                    if (next_weight_vec_dict != weighted_unique_weight_vec_dict):
                      del weighted_unique_weight_vec_dict[weight_vector_tuple]

                  for weight_vector_tuple in non_match_dict:
                    final_non_match_weight_vector_dict[weight_vector_tuple] = \
                                                            non_match_dict[weight_vector_tuple]
                    del next_weight_vec_dict[weight_vector_tuple]
                    if (next_weight_vec_dict != weighted_unique_weight_vec_dict):
                      del weighted_unique_weight_vec_dict[weight_vector_tuple]

                  print 'Deleted %d weight vectors (classified by oracle) from cluster' % \
                        (len(match_dict)+len(non_match_dict))
                  print

                  oracle_time = time.time() - start_time
                  loop_oracle_time_list.append(oracle_time)

                  # Step 2f: Decide if cluster needs/can be split further or not
                  # Stopping criteria:
                  # 1) Cluster is pure enough and not too large (<= max_cluster_size)
                  #    -> Add to final training data
                  # 2) Cluster is too small for further splitting -> Do not use for training
                  # 3) No more budget left for future 'manual' oracle classification

                  # If the cluster is pure enough and not too large then add all its weight
                  # vectors to the final dictionaries of training data, and remove them from
                  # the original weight vector dictionary
                  #
                  if ((cluster_size <= max_cluster_size) and (purity >= min_purity)):
                    print 'Cluster is pure enough and not too large, add its ' + \
                          '%d weight vectors to:' % (cluster_size)

                    num_clusters_used += 1
                    cluster_use_pure_list.append(purity)

                    if (len(match_dict) > len(non_match_dict)):  # The cluster contains matches
                      print '  Match training set'
                      for weight_vector_tuple in next_weight_vec_dict.keys():
                        final_match_weight_vector_dict[weight_vector_tuple] = \
                                                     next_weight_vec_dict[weight_vector_tuple]
                        del weighted_unique_weight_vec_dict[weight_vector_tuple]
                    else:  # The cluster contains non-matches
                      print '  Non-match training set'
                      for weight_vector_tuple in next_weight_vec_dict.keys():
                        final_non_match_weight_vector_dict[weight_vector_tuple] = \
                                                     next_weight_vec_dict[weight_vector_tuple]
                        del weighted_unique_weight_vec_dict[weight_vector_tuple]
                    print

                  # Check if the cluster can be split further
                  #
                  elif (cluster_size <= min_cluster_size):
                    print 'Cluster is too small for further splitting, but not pure enough ' \
                          + 'for further splitting, so do not add to training data'
                    print

                  else:  # The cluster is too large or not pure enough and it can be split
                    print 'Cluster not pure enough or too large, and can be split further'
                    print

                    # Check if we still have manual classifications left
                    #
                    if (len(oracle_class_cache_set) >= budget_num_class):
                      print 'Reached end of manual classification budget'
                      print
                      break  # Leave while loop, no more budget to do manual classification

                    # Step 2d: Split the cluster using a binary classifier
                    #
                    if ((len(match_dict) > 0) and (len(non_match_dict) > 0)):
                      # We need training weight vectors in both classes

                      start_time = time.time()

                      if (split_classifier == 'knn'):
                        class_match_dict, class_non_match_dict = \
                                          knn_split_classifier(next_weight_vec_dict, KNN_K, \
                                                               match_dict, non_match_dict)
                      elif (split_classifier == 'dtree'):
                        class_match_dict, class_non_match_dict = \
                                   dtree_split_classifier(next_weight_vec_dict, match_dict, \
                                                          non_match_dict, num_weights)
                      else:
                        class_match_dict, class_non_match_dict = \
                                   svm_split_classifier(next_weight_vec_dict, match_dict, \
                                                        non_match_dict, num_weights)

                      class_time = time.time() - start_time
                      loop_class_time_list.append(class_time)

                      # First check that both sub-clusters contain weight vectors, if not (i.e.
                      # if all weight vectors are in one sub-cluster) then we cannot add as we
                      # would add the same cluster as we had before -> endless loop
                      #
                      if ((len(class_match_dict) > 0) and (len(class_non_match_dict) > 0)):

                        # Only add to queue if a sub-cluster contains enough weight vectors
                        # (i.e. more than needed in the selection process)
                        #
                        if (len(class_match_dict) > 0):
                          select_num = get_sample_size(len(class_match_dict), \
                                                       cluster_est_proportion, \
                                                       sample_error)
                          if (len(class_match_dict) >= select_num):
                            cluster_queue.append((class_match_dict, purity, entropy,
                                                  len(class_match_dict),
                                                  cluster_est_proportion))
                          else:
                            print '  Match cluster not large enough for required sample size'

                        if (len(class_non_match_dict) > 0):
                          select_num = get_sample_size(len(class_non_match_dict), \
                                                       cluster_est_proportion, \
                                                       sample_error)
                          if (len(class_non_match_dict) > select_num):
                            cluster_queue.append((class_non_match_dict, purity, entropy,
                                                  len(class_non_match_dict),
                                                  cluster_est_proportion))
                          else:
                            print '  Non-match cluster not large enough for required ' + \
                                  'sample size'

                      else:
                        # cluster not used further on in training

                        # This case happens if we have a pure cluster that is too large
                        # It will likely not be split further - so what can we do here?
                        # Split randomly into 2? Do fathest first with k=2, then split into 2
                        # acording to nearest?
                        pass  # TODO **********************************************************

                    else:  # We have training weight vectors in one class only
                      pass # TODO - what here?

                dirDest = "../csv/conjuntosDS/treinoTeste/"+base+"/"+qp+"/PtChr/"
        #         dirDest = "C:/Users/Diego/Documents/NetBeansProjects/Master-SKYAM/AS/src/csv/conjuntosDS/treinoTeste/"
        #         dirDest = "../../Documents/NetBeansProjects/Master-SKYAM/AS/src/csv/conjuntosDS/treinoTeste/"


        #         geraTrainSet(weight_vector_dict_orig, dirDest, 'train' + '(' + int(algUtl) + ')' + num + '.csv')    

        #         geraTestSet(weight_vector_dict_orig, dirDest, 'test' + '(' + int(algUtl) + ')' + num + '.csv')

        #         print 'Number of manual oracle classifications done: %d (out of total ' % \
        #       (len(oracle_class_cache_set)) + 'budget of %d)' % (budget_num_class)
        #         print ''

                abordagem = 'DS'
                #print 'abordagem é %s' %(abordagem)

                #algUtl = linhaAtual['algoritmosUtilizados'].item()
                iteracao = 1
                inspecoesManuais = len(oracle_class_cache_set)
                print linhaAtual['da'].item()

                da = linhaAtual['da'].item()
                dm = len(final_match_weight_vector_dict)
                ndm = len(final_non_match_weight_vector_dict)

                tp = float(linhaAtual['tp'].item() + dm)
                fp = float(linhaAtual['fp'].item())
                tn = float(linhaAtual['tn'].item())# + ndm) #Retirado
                fn = float(linhaAtual['fn'].item() - dm) #Adicionado

        #         print 'tp'
        #         print type(tp)
        #         print tp
        #         print 'fp'
        #         print type(fp)
        #         print fp
        #         print 'tn'
        #         print type(tn)
        #         print tn
        #         print 'fn'
        #         print type(fn)
        #         print fn

                precision = tp/(tp+fp)
        #         print type(precisao)
        #         print precisao
        #         print 'Precisão:'
        #         print type(precision)
        #         print precision
                recall = tp/(tp+fn)
                fmeasure = 2*((precision*recall)/(precision+recall))



                #Adicionando valor à última linha
                estatisticas.loc[(algUtl, etapa, permutacao), ['abordagem', 'iteracao', 'inspecoesManuais',
                   'precision', 'recall', 'f-measure', 'da', 'dm', 'ndm', 'tp',
                   'fp', 'tn', 'fn'] ] = ([abordagem, iteracao, inspecoesManuais,
                   precision, recall, fmeasure, da, dm, ndm, tp, fp, tn, fn])

#                 dirDest = "../csv/conjuntosDS/treinoTeste/"
#                 dirDest = "../csv/conjuntosDS/treinoTeste/"+base+"/"+qp+"/"+"treinoTeste/"
        #         dirDest = "../../Documents/NetBeansProjects/Master-SKYAM/AS/src/csv/conjuntosDS/treinoTeste/"
        #         dirDest = "./arqResult/csv/conjuntosDS/conjuntosDiverg/treinoTeste/"

                #algUtl = str(algUtl).replace('.0','')
                algUtl = str(algUtl)

                geraTrainSet(weight_vector_dict_orig, dirDest, 'train' + '(' + algUtl + ')' + num + '.csv')    

                geraTestSet(weight_vector_dict_orig, dirDest, 'test' + '(' + algUtl + ')' + num + '.csv')

                #Para voltar o dataframe ao normal (Depois organizar as colunas)

        estatisticas = estatisticas.reset_index(level=['algoritmosUtilizados', 'etapa', 'permutacao'])

        estatisticas = estatisticas[['abordagem', 'etapa', 'algoritmosUtilizados', 'permutacao', 'iteracao', 'inspecoesManuais', 'precision', 'recall', 'f-measure', 'da', 'dm', 'ndm', 'tp', 'fp', 'tn', 'fn']]

        estatisticas[['algoritmosUtilizados', 'iteracao', 'inspecoesManuais', 'da', 'dm', 'ndm', 'tp', 'fp', 'tn', 'fn']] = \
        estatisticas[['algoritmosUtilizados', 'iteracao', 'inspecoesManuais', 'da', 'dm', 'ndm', 'tp', 'fp', 'tn', 'fn']].astype(int)

        # Diretório para Windows
#         dirEst = "../csv/"
        # dirEst = "C:\Users\Diego\Documents\NetBeansProjects\Master-SKYAM\AS\src\csv\\"
        # dirEst = "../../Documents/NetBeansProjects/Master-SKYAM/AS/src/csv/"


        # Diretório para Linux
        # dirEst = "./arqResult/csv/"

        estatisticas.to_csv(dirEstat+'estatisticaDS2-PtChr.csv', sep=';', index=False)

Base atual: DBLP2-ACM
QP atual: qp1
Index([u'abordagem', u'iteracao', u'inspecoesManuais', u'precision', u'recall',
       u'f-measure', u'da', u'dm', u'ndm', u'tp', u'fp', u'tn', u'fn'],
      dtype='object')
estatisticas.shape
(3, 13)
Analisando o arquivo: diverg(10)1_NEW.csv

Load weight vector file: ../csv/conjuntosDS/conjuntosDivergAA/DBLP2-ACM/qp1/diverg(10)1_NEW.csv
  Weights to use: ['title', 'authors', 'venue', 'year']
  Number of weight vectors: 4309
    Number of entity ID pairs that occurred more than once: 0

Analyse set of 4309 weight vectors
  Containing 1878 true matches and 2431 true non-matches
    (43.58% true matches)
  Identified 3499 unique weight vectors
  Frequency distribution of occurences of weight vectors:
    Occurence : Number of weight vectors that occur that often
          1 :  3202  (91.51%)
          2 :   201  (5.74%)
          3 :    42  (1.20%)
          4 :    21  (0.60%)
          5 :     7  (0.20%)
          6 :     5  (0.14%)
          7 :     

  The selected farthest weight vectors are:
    [0.750, 0.500, 0.368, 1.000] (False)
    [0.787, 0.702, 0.613, 0.875] (False)
    [0.523, 0.601, 0.452, 1.000] (False)
    [0.702, 0.534, 0.458, 0.875] (False)
    [0.672, 0.711, 0.723, 1.000] (True)
    [0.519, 0.667, 0.452, 0.875] (False)
    [0.518, 0.563, 0.723, 1.000] (False)
    [0.716, 0.544, 0.458, 1.000] (False)
    [0.500, 1.000, 0.452, 0.875] (False)
    [0.817, 0.500, 0.458, 1.000] (False)
    [0.613, 0.607, 0.613, 1.000] (False)
    [0.643, 0.900, 0.458, 0.875] (False)
    [0.661, 0.684, 0.613, 0.875] (False)
    [0.680, 0.697, 0.553, 1.000] (False)
    [0.746, 0.719, 0.380, 0.875] (False)
    [0.747, 0.562, 0.723, 0.875] (False)
    [0.684, 1.000, 0.723, 1.000] (True)
    [0.963, 0.523, 0.553, 0.875] (False)
    [0.629, 0.510, 0.723, 1.000] (False)
    [0.789, 0.625, 0.723, 1.000] (True)
    [0.575, 0.635, 0.553, 0.875] (False)
    [0.518, 0.500, 0.553, 0.875] (False)
    [0.667, 0.657, 0.452, 0.875] (False)
    [0.742, 0.84

Deleted 94 weight vectors (classified by oracle) from cluster

Cluster not pure enough or too large, and can be split further

SVM classification of 8172 weight vectors
  Based on 19 matches and 75 non-matches
  Classified 0 matches and 8172 non-matches

148.0


C:\Users\Diego\Anaconda3\envs\python2\lib\site-packages\ipykernel\kernelbase.py:399: PerformanceWarning: indexing past lexsort depth may impact performance.
  user_expressions, allow_stdin)


Analisando o arquivo: diverg(20)1_NEW.csv

Load weight vector file: ../csv/conjuntosDS/conjuntosDivergAA/DBLP2-ACM/qp1/diverg(20)1_NEW.csv
  Weights to use: ['title', 'authors', 'venue', 'year']
  Number of weight vectors: 9760
    Number of entity ID pairs that occurred more than once: 0

Analyse set of 9760 weight vectors
  Containing 2119 true matches and 7641 true non-matches
    (21.71% true matches)
  Identified 8446 unique weight vectors
  Frequency distribution of occurences of weight vectors:
    Occurence : Number of weight vectors that occur that often
          1 :  7865  (93.12%)
          2 :   385  (4.56%)
          3 :    96  (1.14%)
          4 :    41  (0.49%)
          5 :    17  (0.20%)
          6 :    12  (0.14%)
          7 :     6  (0.07%)
          8 :     7  (0.08%)
          9 :     1  (0.01%)
         10 :     5  (0.06%)
         12 :     2  (0.02%)
         13 :     1  (0.01%)
         14 :     1  (0.01%)
         17 :     1  (0.01%)
         18 :     2  (0